In [3]:
import sys
import numpy as np
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models
import gensim
import csv
import _pickle as cPickle
from sklearn.externals import joblib
from string import digits
import bz2
import pyLDAvis
import pyLDAvis.gensim
'''
Thank you to Jordan Barber's Latent Dirchlet Allocation (LDA) with Python blog. Some of his code from his tutorial
is used here.  https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

Also thanks much to Ben Mabey for creating the pyLDAvis package in python that is used here as well. 
''' 

"\nThank you to Jordan Barber's Latent Dirchlet Allocation (LDA) with Python blog. Some of his code from his tutorial\nis used here.  https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html\n\nAlso thanks much to Ben Mabey for creating the pyLDAvis package in python that is used here as well. \n"

In [ ]:
with open(sys.argv[1]) as f:
    data = f.readlines()

lines = []
for num in data:
    if type(num) != float:
        lines.append(num.replace("_"," "))

newlines = []
for line in lines:
    newlines.append(line.translate({ord(k): None for k in digits}))

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')


texts = []
for i in newlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    
    # add tokens to list
    texts.append(stopped_tokens)
    
#Clean data. Some tokenized words end up as just "s" as the word. 
#For example baby's could be split into 'baby', and 's'. 
i = 0
while i < 5:
    i +=1
    for sentence in texts:
        for word in sentence:
            if word  == 's':f
                sentence.remove(word)


In [ ]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = gensim.models.ldamulticore.LdaMulticore(corpus, id2word=dictionary, 
                                              num_topics=200, chunksize=1000, passes=20, workers=30)


tokens_after_lemmas_and_rm_stopwords = open('tokens_after_lemmas_and_rm_stopwords.txt', 'w')
for item in texts:
    tokens_after_lemmas_and_rm_stopwords.write("%s\n" % item)
    
dictionary.save_as_text('lemmas_nostopwords_with_otherdatacleaning_dictionary_' + sys.argv[2] + '.txt')

corpora.MmCorpus.serialize('lemmas_nostopwords_corpus_'+ sys.argv[2] +'.mm', corpus)
    
joblib.dump(lda, 'ldamodel_'+ sys.argv[2]+ '.pkl')


In [6]:
print(corpus[56])

[(12, 2.0), (26, 1.0), (28, 1.0), (45, 1.0), (50, 1.0), (176, 1.0), (206, 1.0), (281, 1.0), (368, 1.0), (459, 1.0), (538, 1.0), (574, 2.0), (580, 1.0), (613, 1.0), (731, 1.0), (732, 1.0), (733, 1.0), (734, 1.0), (735, 1.0), (736, 1.0), (737, 2.0), (738, 1.0)]


In [4]:
dictionary = gensim.corpora.Dictionary.load_from_text('lemmas_nostopwords_with_otherdatacleaning_dictionary_1000000.txt')
corpus = gensim.corpora.MmCorpus('lemmas_nostopwords_corpus_1000000.mm')
lda = joblib.load('ldamodel_1000000.pkl')

(lda.print_topics(num_topics=20, num_words=8))

[(128,
  '0.135*bra + 0.053*cup + 0.047*support + 0.037*strap + 0.022*underwire + 0.019*adjustable + 0.017*comfort + 0.016*shape'),
 (89,
  '0.248*neck + 0.124*v + 0.083*crew + 0.076*sleeve + 0.035*tee + 0.033*cotton + 0.028*short + 0.019*machine'),
 (198,
  '0.093*men + 0.050*cotton + 0.033*gray + 0.028*man + 0.027*make + 0.024*shirt + 0.023*percent + 0.023*gingham'),
 (20,
  '0.293*lace + 0.065*trim + 0.059*sheer + 0.031*cami + 0.017*floral + 0.017*top + 0.016*pretty + 0.016*scallop'),
 (116,
  '0.082*movement + 0.042*freedom + 0.038*allow + 0.033*knee + 0.028*crotch + 0.026*range + 0.025*pant + 0.024*fabric'),
 (125,
  '0.110*club + 0.059*eur + 0.051*princess + 0.046*room + 0.019*dash + 0.019*panache + 0.018*rabbit + 0.017*slipper'),
 (63,
  '0.033*size + 0.033*review + 0.024*fit + 0.020*bust + 0.020*email + 0.020*read + 0.019*address + 0.017*full'),
 (88,
  '0.121*spring + 0.095*coral + 0.017*bee + 0.016*columbia + 0.015*filament + 0.014*creamy + 0.013*two + 0.012*make'),
 (49,
  '

In [4]:
lda_vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(lda_vis)